## 什么是JVM？
- JVM 是 Java Virtual Machine
- JVM 本质上就是一个软件，是计算机硬件的一层软件抽象
- JAVA 在编译后会生成类似于汇编语言的 JVM 字节码，字节码是在 JVM 上跑，需要由 JVM 把字节码翻译成机器指令，才能使 JAVA 程序跑起来
- C 语言编译后产生的汇编语言会直接在硬件上跑
- 根据 JVM 规范实现的具体虚拟机有几十种，主流的 JVM 包括 Hotspot、Jikes RVM 等，都是用 C/C++ 和汇编编写的。每个 JRE 编译的时候针对每个平台编译，因此下载 JRE（JVM、Java核心类库和支持文件）的时候是分平台的，JVM 的作用是把平台无关的 .class 里面的字节码翻译成平台相关的机器码，来实现跨平台。

---

## 什么是 DVM，和 JVM 有什么不同？
- DVM：Dalvik Virtual Machine，是安卓中使用的虚拟机
- 所有安卓程序都运行在安卓系统进程里，每个进程对应着一个 Dalvik 虚拟机实例
- 他们都提供了对象生命周期管理、堆栈管理、线程管理、安全和异常管理以及垃圾回收等重要功能，各自拥有一套完整的指令系统
- DVM 运行的是 Dalvik 字节码，所有的 Dalvik 字节码由 JAVA 字节码转换而来，并被打包到一个 DEX（Dalvik Executable）可执行文件中，DVM 通过解释 DEX 文件来执行这些字节码
- Dalvik 可执行文件体积更小
  - 下图是 JVM 规范中以 C 的数据结构表达的 class 文件结构，class 文件被虚拟机加载到内存中后便是这样
  - class 文件中包含多个不同的方法签名，如果 A 类文件引用 B 类文件中的方法，方法签名也会被复制到 A 类文件中（在虚拟机加载类的连接阶段将会使用该签名链接到 B 类的对应方法）
  
![image](page1.png)


- 为了减小执行文件的体积，安卓使用 Dalvik 虚拟机，SDK 中有个 dx 工具负责将 JAVA 字节码转换为 Dalvik 字节码，dx 工具对 JAVA 类文件重新排列，将所有 JAVA 类文件中的常量池分解，消除其中的冗余信息，重新组合形成一个常量池，所有的类文件共享同一个常量池，使得相同的字符串、常量在 DEX 文件中只出现一次，从而减小了文件的体积

![image](page2.png)

- JVM 基于栈，DVM 基于寄存器
  - JAVA 虚拟机基于栈结构，程序在运行时虚拟机需要频繁的从栈上读取写入数据，这个过程需要更多的指令分派与内存访问次数，会耗费很多 CPU 时间
  - Dalvik 虚拟机基于寄存器架构，数据的访问通过寄存器间直接传递，这样的访问方式比基于栈方式要快很多
  - 备注：相比 class 文件，编译后的 dex 文件代码指令减少了，运行速度就更快了

![image](page3.png)

---

## 什么是 ART 虚拟机，和 JVM/DVM 有什么不同？
- 首先了解 JIT（Just In Time，即时编译技术）和 AOT(Ahead Of Time，预编译技术)两种编译模式
  - JIT 以 JVM 为例，javac 把程序源码编译成 JAVA 字节码，JVM 通过逐条解释字节码将其翻译成对应的机器指令，逐条读入，逐条解释翻译，执行速度必然比 C/C++ 编译后的可执行二进制字节码程序慢，为了提高执行速度，就引入了 JIT 技术，JIT 会在运行时分析应用程序的代码，识别哪些方法可以归类为热方法，这些方法会被 JIT 编译器编译成对应的汇编代码，然后存储到代码缓存中，以后调用这些方法时就不用解释执行了，可以直接使用代码缓存中已编译好的汇编代码。这能显著提升应用程序的执行效率。（安卓 Dalvik 虚拟机在 2.2 中增加了 JIT）
  - AOT 就是指 C/C++ 这类语言，编译器在编译时直接将程序源码编译成目标机器码，运行时直接运行机器码

- Dalvik 虚拟机执行的是 dex 字节码，ART 虚拟机执行的是本地机器码
  - Dalvik 执行的是 dex 字节码，依靠 JIT 编译器去解释执行，运行时动态地将执行频率很高的 dex 字节码翻译成本地机器码，然后在执行，但是将 dex 字节码翻译成本地机器码是发生在应用程序的运行过程中，并且应用程序每一次重新运行的时候，都要重新做这个翻译工作，因此，即使采用了 JIT，Dalvik 虚拟机的总体性能还是不能与直接执行本地机器码的 ART 虚拟机相比
  - 安卓运行时从 Dalvik 虚拟机替换成 ART 虚拟机，并不要求开发者重新将自己的应用直接编译成目标机器码，也就是说，应用程序仍然是一个包含 dex 字节码的 apk 文件
  - 在安装应用的时候，dex 中的字节码将被编译成本地机器码，之后每次打开应用，执行的都是本地机器码。移除了运行时的解释执行，效率更高，启动更快

---

## 总结
### ART 优点：
1. 系统性能显著提升
2. 应用启动更快、运行更快、体验更流畅、触感反馈更及时
3. 续航能力提升
4. 支持更低的硬件

### ART 缺点
1. 更大的存储空间占用，可能增加 10% - 20%
2. 更长的应用安装时间

---